In [1]:
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document
import json
import os
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [3]:
import json

# Data entries (replace with your own objects as needed)
data = [
    {
        "id": None,
        "metadata": {"source": "./txt_docs/anomaly_calculation.txt"},
        "page_content": "This code computes sea surface temperature anomalies by removing long-term trends and seasonal cycles through harmonic regression. It is useful for climate and oceanography tasks where highlighting interannual or short-term variability is needed after filtering out predictable signals. Packages used: numpy, pandas, matplotlib, xarray, scipy.signal",
        "type": "Document"
    },
    {
        "id": None,
        "metadata": {"source": "./txt_docs/arctic_ocean_plot.txt"},
        "page_content": "This code generates a polar stereographic map of the Arctic, displaying sea surface temperature data with coastlines, land shading, and gridlines for geographic context. It is useful for climate and oceanography tasks focused on visualizing spatial variability in polar regions with accurate projections. Packages used: cartopy, matplotlib, xarray, numpy",
        "type": "Document"
    },
    {
        "id": None,
        "metadata": {"source": "./txt_docs/atlantic_plot.txt"},
        "page_content": "This code creates a Plate Carrée projection map of the Atlantic Ocean and visualizes sea surface temperature data with geographic context. It is useful for climate and oceanography tasks that require mapping large ocean basins, enabling clear interpretation of spatial variability across latitudes and longitudes. Packages used: cartopy, matplotlib, xarray, numpy",
        "type": "Document"
    },
    {
        "id": None,
        "metadata": {"source": "./txt_docs/filters.txt"},
        "page_content": "This code applies Butterworth filters to a sea surface temperature time series to isolate variability on different timescales. It computes low-pass (~30 d), high-pass (~7 d), and band-pass (10–90 d) signals and visualizes them alongside the original data using dual axes. It is useful for climate and oceanography tasks that require separating short-term, intraseasonal, and longer-term variability, supporting analyses of anomalies, oscillations, and filtering prior to further statistical methods. Packages used: numpy, pandas, matplotlib, xarray, scipy.signal",
        "type": "Document"
    },
    {
        "id": None,
        "metadata": {"source": "./txt_docs/rolling_mean.txt"},
        "page_content": "This code smooths a sea surface temperature time series using rolling means over 7-day and 30-day windows and compares them to the original data. It is useful for climate and oceanography tasks where short-term fluctuations need to be filtered out to highlight weekly or monthly variability and trends. Packages used: numpy, pandas, matplotlib, xarray",
        "type": "Document"
    },
    {
        "id": None,
        "metadata": {"source": "./txt_docs/southern_ocean_plot.txt"},
        "page_content": "This code creates a polar stereographic map of the Southern Ocean, plotting sea surface temperature data with coastlines, land shading, and gridlines for geographic reference. It is useful for climate and oceanography tasks focused on visualizing spatial variability around Antarctica with an appropriate polar projection. Packages used: cartopy, matplotlib, xarray, numpy",
        "type": "Document"
    }
]

In [14]:
docs = []
for doc in data:
    # use the definition as the content, and keep the term (and letter) as metadata
    docs.append(
        Document(
            page_content=doc['page_content'],
            metadata=doc['metadata']
        )
    )

In [19]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings
doc_embedder = HuggingFaceEndpointEmbeddings(
    model="Qwen/Qwen3-Embedding-8B",
    task="feature-extraction",
    model_kwargs={"normalize": True},
    huggingfacehub_api_token = hf_token
    
)

In [20]:
vector_store_hf = Chroma.from_documents(docs, doc_embedder)
retriever = vector_store_hf.as_retriever()

In [27]:
def get_example_of_visualizations(query: str) -> str:
    """
    Retrieves example visualizations related to the query.

    Parameters:
    - query (str): The user's query about plotting.

    Returns:
    - str: The content of the most relevant example file.
    """


    # Perform a similarity search
    results = vector_store_hf.similarity_search_with_score(query, k=1)

    # Extract the most relevant document
    doc, score = results[0]

    # Construct the full path to the txt file
    file_name = doc.metadata['source'].lstrip('./')
    full_path = os.path.join('./', file_name)

    # Read and return the content of the txt file
    try:
        with open(full_path, 'r', encoding='utf-8') as file:
            content = file.read()
            return content
    except Exception as e:
        print(f"An error occurred while reading the file: {str(e)}")
        return ""  # Return empty string if error occurs


In [30]:
get_example_of_visualizations('one year rolling mean for salinity')

'import numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport xarray as xr\n\nFILE_IN = "text.nc"\n\nds = xr.open_dataset(FILE_IN)\ntime = pd.to_datetime(ds["time"].values)\ny = pd.Series(ds["analysed_sst"].values, index=time).astype(float)\ny = y.interpolate(limit_direction="both")\n\nrun7  = y.rolling(window=7,  center=True, min_periods=1).mean()\nrun30 = y.rolling(window=30, center=True, min_periods=1).mean()\n\n\n# 1) Original & running means\nplt.figure(figsize=(12,4.5))\nplt.plot(y.index, y.values, label="Original")\nplt.plot(run7.index, run7.values, label="7-day running mean")\nplt.plot(run30.index, run30.values, label="30-day running mean")\nplt.title("SST: Original & Running Means")\nplt.xlabel("Date")\nplt.ylabel("Temperature")\nplt.legend()\nplt.tight_layout()\nplt.show()'